In [5]:
import os
import math
import numpy as np
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import os
import re
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [17]:
def normalize(file_str):
    # 1) remove numbers and spacial characters
    file_str = re.sub(r'([^a-zA-Z\s]+?)', '', file_str).replace("\n",' ')
    # 2) Lower case
    file_str = file_str.lower()
    return file_str

In [18]:
# Reads all the files of a corpus into a string variable. The corpus path and list of folders to be scanned can be specified
def read_corpus(path, folder_list):
    corpus_lst = []
    corpus_str = ""
    for folder in os.listdir(corpus_path):
        if folder in folder_list:
            for file in os.listdir(os.path.join(corpus_path,folder)):
                with open (os.path.join(corpus_path,folder,file), "r") as myfile:
                    file_data = "".join(myfile.readlines())
                    normalized_file_data = normalize(file_data)
                    corpus_lst.append(normalized_file_data)
    return corpus_lst

In [19]:
corpus_path = os.path.join(os.getcwd(),"HW2","20_newsgroups")
folder_list = ['alt.atheism']
corpus_lst = read_corpus(corpus_path, folder_list)

In [22]:
# Output corpus into text file
filename = 'HW2//corpus_output.txt'

with open(filename, mode="w") as outfile: 
    for s in corpus_lst:
        outfile.write("%s\n" % s)

In [6]:
path = get_tmpfile("word2vec.model")

In [26]:
# Get corpus as a list of lists
corpus = []
for sentence in corpus_lst:
    # word tokenization
    corpus.append(word_tokenize(sentence))

In [27]:
# train word2vec model
model = Word2Vec(corpus, size=100, window=5, min_count=5, workers=4, sg=1)
model.save("word2vec.model")

In [29]:
emb_vec = model.wv['student']
emb_vec

array([ 1.48345763e-02, -7.56442472e-02,  3.07667077e-01, -1.56090453e-01,
        1.51485845e-01, -1.86642200e-01, -1.85613084e-04,  7.50138331e-03,
        4.16409075e-01, -1.51707912e-02, -9.81734842e-02,  1.20306149e-01,
       -6.83321282e-02,  1.21813260e-01,  1.38223931e-01, -1.14426345e-01,
        8.08655692e-04, -2.87239961e-02, -5.02260216e-02,  2.06014365e-01,
        7.63981417e-02,  1.92871336e-02,  1.86544925e-01, -2.85521030e-01,
       -2.59806383e-02, -1.16892666e-01,  1.19654514e-01, -4.14459109e-02,
       -5.70675619e-02, -1.89218894e-01,  1.86056480e-01, -1.37155708e-02,
       -7.68367127e-02,  9.55482125e-02,  1.06406204e-01,  1.50114387e-01,
        8.17096233e-02, -1.05754510e-01, -1.15877509e-01,  2.43294854e-02,
       -5.14528975e-02,  2.74628818e-01,  2.03077197e-01, -3.30013260e-02,
        8.13637152e-02, -1.59350410e-01, -7.87228569e-02,  1.17078580e-01,
       -2.80741185e-01,  1.44151747e-01, -2.56304443e-02,  1.07414722e-01,
       -1.07750082e-02,  

In [34]:
# Similar words
print(model.wv.most_similar('take')[:5])

[('give', 0.8342376947402954), ('straighten', 0.8339373469352722), ('attempt', 0.824309229850769), ('continue', 0.8242608308792114), ('put', 0.819233775138855)]


In [41]:
print(model.wv.most_similar('email')[:5])

[('send', 0.9342182278633118), ('mail', 0.8977521061897278), ('complaints', 0.8578709363937378), ('please', 0.8435264825820923), ('consisting', 0.8429056406021118)]


## hw2) 
### Q1)

#### Skip Gram - 'faith'

In [49]:
window_sizes = [5,20,50,75]
sizes = [50,100,150,200]
for size in sizes:  
    for window in window_sizes:
        model = Word2Vec(corpus, size=size, window=window, min_count=5, workers=4, sg=1)
        print('size=',size,', window=',window,':', model.wv.most_similar('faith')[:5])
        print("**********************************************************************")

size= 50 , window= 5 : [('dogma', 0.886344850063324), ('atheism', 0.8285374045372009), ('strong', 0.804130494594574), ('weak', 0.8038588762283325), ('belief', 0.7770904898643494)]
**********************************************************************
size= 50 , window= 20 : [('dogma', 0.8696205019950867), ('dangerous', 0.7451220750808716), ('religion', 0.7171891927719116), ('leap', 0.7090973258018494), ('flying', 0.6973340511322021)]
**********************************************************************
size= 50 , window= 50 : [('dogma', 0.8624674081802368), ('flying', 0.765070378780365), ('irrationality', 0.7509381175041199), ('apredtneatcstorontoedu', 0.7399941682815552), ('tgkcstorontoedu', 0.7345501184463501)]
**********************************************************************
size= 50 , window= 75 : [('dogma', 0.8395116329193115), ('todd', 0.7802084684371948), ('tgkcstorontoedu', 0.7708868980407715), ('irrationality', 0.766791820526123), ('kelley', 0.7593604922294617)]
********

We see that the results are more consistent across different configurations, the closest word is always dogma. This makes sense and seems correct.

At larger window sizes, irrelevant words are included in the context and hence are considered similar to the work of interest, eg. window = 50, word 'apredtneatcstorontoedu' is considered similar to 'faith'. However is purely an artefact of our corpus.

The larger the embedding size, the more complex the model and the longer the time to generate the embeddings, however assuming we have a sufficiently large corpus, larger embedding size can mean better overall performance of the final model.


#### Cbow  - 'faith'

In [50]:
window_sizes = [5,20,50,75]
sizes = [50,100,150,200]
for size in sizes:  
    for window in window_sizes:
        model = Word2Vec(corpus, size=size, window=window, min_count=5, workers=4, sg=0)
        print('size=',size,', window=',window,':', model.wv.most_similar('faith')[:5])
        print("**********************************************************************")

size= 50 , window= 5 : [('reason', 0.9777678847312927), ('possible', 0.9760324358940125), ('nature', 0.9744144678115845), ('subjective', 0.9729087948799133), ('religion', 0.9710952043533325)]
**********************************************************************
size= 50 , window= 20 : [('statement', 0.9907295107841492), ('logically', 0.9872539639472961), ('reason', 0.9713360667228699), ('gods', 0.9711054563522339), ('exists', 0.970790445804596)]
**********************************************************************
size= 50 , window= 50 : [('proof', 0.9398975372314453), ('existence', 0.9395597577095032), ('gods', 0.9389523267745972), ('proves', 0.9328867197036743), ('agnosticism', 0.9265393018722534)]
**********************************************************************
size= 50 , window= 75 : [('proof', 0.9480326771736145), ('existence', 0.9271202683448792), ('dogma', 0.9200506210327148), ('strong', 0.9175812005996704), ('atheism', 0.9134929776191711)]
******************************

### Skip gram- 'church'

In [52]:
window_sizes = [5,20,50,75]
sizes = [50,100,150,200]
for size in sizes:  
    for window in window_sizes:
        model = Word2Vec(corpus, size=size, window=window, min_count=5, workers=4, sg=1)
        print('size=',size,', window=',window,':', model.wv.most_similar('church')[:5])
        print("**********************************************************************")

size= 50 , window= 5 : [('fathers', 0.8552189469337463), ('constitution', 0.8520639538764954), ('moon', 0.8507142663002014), ('priest', 0.8500820398330688), ('nations', 0.8499590754508972)]
**********************************************************************
size= 50 , window= 20 : [('campaign', 0.7513174414634705), ('foundation', 0.7351351976394653), ('rs', 0.7272127270698547), ('baptist', 0.7142115831375122), ('president', 0.7027206420898438)]
**********************************************************************
size= 50 , window= 50 : [('smith', 0.6751267313957214), ('judaism', 0.663597822189331), ('christ', 0.6380735635757446), ('saint', 0.6342097520828247), ('churchs', 0.6340110301971436)]
**********************************************************************
size= 50 , window= 75 : [('smith', 0.7112983465194702), ('major', 0.7006593942642212), ('churchs', 0.6824931502342224), ('churches', 0.6723576784133911), ('jr', 0.6599758863449097)]
****************************************

### CBOW- 'church'

In [ ]:
window_sizes = [5,20,50,75]
sizes = [50,100,150,200]
for size in sizes:  
    for window in window_sizes:
        model = Word2Vec(corpus, size=size, window=window, min_count=5, workers=4, sg=0)
        print('size=',size,', window=',window,':', model.wv.most_similar('church')[:5])
        print("**********************************************************************")

Cbow does a better job in that no irrelevent words appear even on larger window sizes, CBOW also performs significantly faster than skip gram. 

however overall we see that skip gram gives us more consistent results across various parameter sizes. This is because skip-grams gives more weightage to words that are closer to the word of interest.